## Global Settings and Imports

In [1]:
# jupyter notebook에서 import 해서 쓰는 모듈의 코드가 변경될 시, 변동 사항을 자동으로 반영해주는 기능 켜기
%load_ext autoreload
%autoreload 2

In [ ]:
from utils.model_manager import ModelManager, ManagerArgs

## Prepare parameters

In [ ]:
args = ManagerArgs(
    config = "config/model/sentirec/vader_manual1.yaml",
    resume_ckpt_path = None
)

## 1. 원클릭 학습

In [ ]:
model_manager = ModelManager(args)

Seed set to 1234
100%|██████████| 3246/3246 [00:00<00:00, 22316.34it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
model_manager.fit()

## 2. 내부 작동 방식 훑어보기

## Load data

## Create model

## Train model

### 기타 테스트용 코드

In [ ]:
# 한 impression 데이터의 history에 포함된
# 모든 뉴스의 제목이 저장된 배치 데이터의 길이.
# history가 설정값보다 더 적을 경우,
# 빈 공간은 padding값(0)으로 채워집니다.
len(model_manager.train_dataset[0]['h_title'])

200

In [ ]:
# 모든 뉴스의 token 길이 또한 고정됩니다.
# 마찬가지로 모든 빈칸은 padding값(0)으로 채워집니다. 
len(model_manager.train_dataset[0]['h_title'][0])

100